**Angelo Joaquin B. Alvarez \(210295\)**


# Homework 2

*Due date:* February 20, 2024 (Tuesday) at 8 PM on CodePost

This homework helps you get a grasp in using SageMath for doing computations in number theory and algebra.
This *should* be a bit easier than the previous homework.

This homework has 30 points in total. Final percentages are capped at 100%.

Please be guided on the policies regarding late submissions, regrading, and collaboration.
If any, please direct all your questions and clarifications about this homework in the `#hw2-help` channel on the Discord server.

## Getting started

SageMath is what you call a *computer algebra system*, a kind of software that can manipulate mathematical
expressions and objects in a way similar to how mathematicans do it manually.

It covers a lot of fields of math (from undergrad to research-level) so it's likely that we'll just
scratch the surface in terms of features for this course.
In particular, we'll use SageMath for its very efficient implementations of number-theoretic algorithms 
and various tools to deal with algebraic structures that are widely used in cryptography.

*Side note:* I also used SageMath (and also Python) for my undergrad thesis, so I should have enough
experience to help you out, so ask away at the `#sagemath-help` channel!

## Some reminders

Before starting, please make sure you have selected the correct kernel.
On the top right corner of the notebook, it should say **SageMath**, not Python 3.

You may find the following pages from the SageMath docs helpful:
* https://doc.sagemath.org/html/en/reference/finite_rings/sage/rings/finite_rings/integer_mod.html
* https://doc.sagemath.org/html/en/thematic_tutorials/numtheory_rsa.html
* https://doc.sagemath.org/html/en/constructions/number_theory.html
* https://doc.sagemath.org/html/en/tutorial/tour_polynomial.html
* https://doc.sagemath.org/html/en/constructions/polynomials.html

## 2-1. Quadratic residues [8 pts]

In the lecture notes, we have seen how to do arithmetic modulo $n$.
Now what does it mean to take the square root modulo an integer?
More concretely: How can Bob tell whether a given number $a$ is equal to a square modulo $p$?

Suppose that Alice asks Bob whether $181$ is a square modulo $1223$.
One way for Bob to answer Alice's question is by constructing a table of squares modulo $1223$:
$$
\begin{array}{r|cccccccc}
c & 1 & 2 & 3 & \cdots & 34 & 35 & 36 & \cdots \\ \hline
c^2 \bmod 1223 & 1 & 4 & 9 & \cdots & 1156 & 2 & 73 & \cdots \\
\end{array}
$$

Bob eventually found that $437^2 \equiv 181 \pmod{1223}$, thus the answer to her question is 
that $181$ is indeed a square modulo $1223$.
Meanwhile, if Alice is sufficiently motivated to continue the table all the way up to 
$1222^2 \bmod 1223$, she can verify that the number $385$ is not a square modulo $1223$, because 
it does not appear in her table.

**Definition.** Let $p$ be an odd prime number and let $a$ be a number with $p \nmid a$
($p$ does not divide $a$).
We say that $a$ is a *quadratic residue modulo $p$* if $a$ is a square modulo $p$, i.e., 
if there is a number $c$ so that $c^2 \equiv a \pmod{p}$.
If $a$ is not a square modulo $p$, i.e., if there does not exist such number $c$, then $a$ 
is called a *quadratic nonresidue modulo $p$*.

Instead of directly computing the square mod $p$ for every possible value just to see whether an integer is a quadratic residue or not, we determine what's called the *Legendre symbol*, which we define as follows:

**Definition.** Let $p$ be an odd prime and let $a$ be an integer. The Legendre symbol of $a$ with respect to $p$ is defined by $$(\tfrac{a}{p}) =
\begin{cases}
    1 & \text{if } a \text{ is a quadratic residue mod } p \text{ and } a \not\equiv 0 \pmod{p} \\
    -1 & \text{if } a \text{ is a quadratic nonresidue mod } p \\
    0 & \text{if } a \equiv 0 \pmod{p}
\end{cases}$$

Computing the Legendre symbol $(\tfrac{a}{p})$ turns out to be straightforward, since we can use a property called *Euler's criterion*:

**Theorem.** Let $p$ be an odd prime and $a$ is a positive integer not divisible by $a$. Then
$$
(\tfrac{a}{p}) = a^{(p-1)/2} \bmod p.
$$



**(a) [5 pts]** Using the information above, implement **your own** function called `is_quadratic_residue` that returns `True` if `a` is a quadratic residue mod `p`, where `a` and `p` satisfy the above constraints.

In [2]:
def is_quadratic_residue(a, p):
    assert is_prime(p), 'p must be prime'
    qr = 0
    for x in range(0, p):
        if mod(x**2, p) == mod(a, p):
            return true
    return false

(*Note:* $0$ is considered a special case, as there's disagreement whether to consider $0$ a quadratic residue for any modulo $p$, but common consensus points towards *no*. So for the purposes of the homework, $0$ is not a quadratic residue.)



**(b) [3 pts]** In the list below, there are four quadratic nonresidues mod $p$ and one quadratic residue mod $p$, where the value of $p$ is also given.
Find the quadratic residue in the list (use your `is_quadratic_residue` function for this purpose) and then calculate its two square roots using the built-in `.sqrt` method.
Yes, there are two of them since if $c^2 \equiv a \pmod{p}$, then $(-c)^2 \equiv a \pmod{p}$.

In [3]:
p = 101
numbers = [42, 56, 7, 63, 91]

In [4]:
for e in numbers:
    if is_quadratic_residue(e, p):
        print(f"The Square Roots are: {mod(e, p).sqrt(all=True)}")

The Square Roots are: [37, 64]


## 2-2. Cyclic redundancy checks [10 pts]

A *cyclic redundancy check* (CRC) is a method used to check data integrity. To transmit some binary data using a CRC, a *check value* is first computed from the data. When the data is received, the check value is computed again using the received data. If the check value matches the one previously calculated, the data is valid. If the check value does not match the one previously calculated, then that indicates the data was corrupted in transit and is invalid.

There are many different implementations of CRCs, so we'll present a simple version. To generate a check value, we first convert the binary data to a polynomial in $\mathbb{Z}_2[x]$, where the coefficients are the bits in the data. For example, to generate the check value for the 8-bit string `10101111`, we write:
$$
\begin{align*}
    \texttt{10101111} &\Rightarrow 1x^7 + 0x^6 + 1x^5 + 0x^4 + 1x^3 + 1x^2 + 1x + 1\\
    &= x^7 + x^5 + x^3 + x^2 + x + 1.
\end{align*}
$$
    We then get the remainder from dividing this polynomial by some generator polynomial. For this example, let's use the generator polynomial $x^3 + x + 1$. This means we are working in the finite field $\mathbb{Z}_2[x]/\langle x^3 + x + 1\rangle$.

**(a) [2 pts]** Show that the remainder when $x^7 + x^5 + x^3 + x^2 + x + 1$ is divided by $x^3 + x + 1$ is $x$.

In [15]:
R.<x> = GF(2)[]
f = x^7 + x^5 + x^3 + x^2 + x + 1
g = x^3 + x + 1
h = f.quo_rem(g)
print(f"quotient: {h[0]}")
print(f"remainder: {h[1]}")

quotient: x^4 + x + 1
remainder: x


Since $x^7 + x^5 + x^3 + x^2 + x + 1 \equiv x \pmod{x^3 + x + 1}$, we then convert the remainder $x$ back into a bit string. Since our generator polynomial has degree 3, our check value must be 3 bits long.
$$
\begin{align*}
    x &= 0x^2 + 1x + 0\\
    &\Rightarrow \texttt{010}.
\end{align*}
$$
Therefore, the correct check value for `10101111` computed using the generator polynomial $x^3 + x + 1$ is `010`.

**(b) [5 pts]** You receive the data `01101100` with CRC check value `111`, using the generator polynomial $x^3 + x + 1$. Is the data valid?



In [8]:
R.<x> = GF(2)[]
f = x^6 + x^5 + x^3 + x^2
g = x^3 + x + 1
h = f.quo_rem(g)
print(f"quotient: {h[0]}\nremainder: {h[1]}\n\nCheck Value: 101\nTherefore it does not have the correct CRC check value.")

quotient: x^3 + x^2 + x + 1
remainder: x^2 + 1

Check Value: 101
Therefore it does not have the correct CRC check value.


In practice, we use larger generator polynomials, so we now have a larger range of check values.
CRC-32 is a cyclic redundancy check algorithm which relies on the finite field $K = \mathbb{Z}_2[x]/\langle g(x)\rangle$, where
$$
    g(x) = x^{32} + x^{26} + x^{23} + x^{22} + x^{16} + x^{12} + x^{11} + x^{10}  + x^{8} + x^{7} + x^{5} + x^{4} + x^{2} + x + 1.
$$

**(c) [3 pts]** Without using SageMath, how many elements does $K$ have? Justify your answer, but *please* do not list down all the elements of $K$.



**Solution:**

Since modulo $g(x)$ has a degree of 32, this means that any polynomial in the quotient ring $\mathbb{Z}_2 [x]$ can be represented in 32 degrees. Additionally, $K$ will contain polynomials that are remainders of any polynomial in the field $\mathbb{Z}_2[x]$ is divided by the modulo polynomial $g(x)$. 

Thus, polynomials within $K$ will have at most 31 degrees. To find every distinct element within $K$, we must find all possible polynomials with degrees ranging from 0 to 31. This can be done by representing each polynomial through:

$$
f(x)=a_{n}x^n +a_{n-1}x^{n-1} + a_{n-2}x^{n-2} + \cdots +a_2 x^2 +a_1x^1 + a_0
$$

where $a_i$ is either 1 or 0 and $n$ is the degree of the polynomial. Since we have at most 32 terms from $a_n$ to $a_0$ and there are only two possible values for $a_i$, we can compute for the total number of possible polynomials within $K$ using:

\begin{align}
|K|&= \underbrace{2 * 2 * 2*\cdots 2 * 2* 2}_{\text{32 times}} \\
&= 2^{32} \\
|K| &= 4294967296 \text{ elements}
\end{align}

Thus, the number of elements within $K$ is $2^{32}$ polynomials, or in the context of CRC\-32, $2^{32}$ _check values_

### The Number of Elements in $K$ is $2^{32}$



## 2-3. Key exchange with group theory [8 pts]



(*Note:* Answer this item manually.)

Alice and Bob want to have a *shared secret*, a piece of information which both of them knows but no one else does.
Fortunately, they both recently learned group theory!

They both decided to use the group $G = \{a,b,c,d,e,f,g,h\}$, defined with the multiplication table below:
$$
\begin{array}{c|cccc|cccc}
    \cdot & a & b & c & d & e & f & g & h \\\hline
        a & a & b & c & d & e & f & g & h \\
        b & b & c & d & a & g & h & f & e \\
        c & c & d & a & b & f & e & h & g \\
        d & d & a & b & c & h & g & e & f \\\hline
        e & e & h & f & g & a & c & d & b \\
        f & f & g & e & h & c & a & b & d \\
        g & g & e & h & f & b & d & a & c \\
        h & h & f & g & e & d & b & c & a \\
\end{array}
$$

Here, $a$ is the identity element (not $e$). For any $x, y \in G$, define the operation $x \diamond y = y^{-1} \cdot x \cdot y$.



**(a) [2 pts]** Verify that $G$ is *not* abelian by picking any two distinct elements $x, y \in G$ such that $x \cdot y \ne y \cdot x$.

**Solution:**

In the multiplication table above, $e \cdot g = d$, while $g \cdot e = b$. Since  $d \neq b$, then $e \cdot g \neq g \cdot e$ therefore the group $G$ is non\-abelian.

### Since $e \cdot g \neq g \cdot e$, then $G$ is non-abelian

**(b) [2 pts]** Find $d^{-1}$, the inverse of $d$.

**Solution:** 

An inverse of an element $d$ in group $G$ is defined as a unique element $d^{-1}$ also in group $G$ such that $d \cdot d^{-1} = d^{-1} \cdot d = a$, where $a$ is the identity element.

Looking at the multiplication table, the only product that results to $a$ in the row and column of $d$ is when it is multiplied to $b$. Thus, we can conclude that $b$ is the inverse of $d$ or:

\begin{align}
&\text{Since } d \cdot d^{-1} = d^{-1} \cdot d = a\\
&\text{and } d \cdot b = b \cdot d = a \\
&\therefore b =d^{-1}
\end{align}

### $\therefore d^{-1} = b$



Their "homebrew" key exchange protocol works like this:

* First, Alice and Bob agree to pick the element $w = e$ to start with. This agreement is performed through a non-secure channel, so it is not a secret that $e$ was chosen for the value of $w$.
    
* Alice then picks $b$ in secret and computes $P_A = w \diamond b$, while Bob independently picks $c$ in secret and computes $P_B = w \diamond c$. 
    
* Then, Alice sends $P_A$ to Bob while Bob sends $P_B$ to Alice. These are also done through a non-secure channel, so neither $P_A$ nor $P_B$ are secret.

**(c) [2 pts]** What are the values of $P_A$ and $P_B$? (the answers must be a letter from $a$ to $h$)



**Solution:** 

Note: the $\diamond$ operation is defined as $x \diamond y = y^{-1} \cdot x \cdot y$.

For $P_A$:

\begin{align}
P_A &= w \diamond b \\
&= b^{-1} \cdot w \cdot b \\
&= d \cdot e \cdot b \\
&= h \cdot b \\
P_A &= f\end{align}

For $P_B$: 

\begin{align} 
P_B &= w \diamond c \\
&= c^{-1} \cdot w \cdot c \\
&= c \cdot e \cdot c \\ 
&= f \cdot c \\
P_B &= e\end{align}

### $\therefore P_A = f$ and $P_B = e$



* Since Alice knows she picked $b$, she then computes $K_A = P_B \diamond b$. Similarly, since Bob knows he picked $c$, he then computes $K_B = P_A \diamond c$.

**(d) [2 pts]** What are the values of $K_A$ and $K_B$? Are they equal?



**Solution:** 

For $K_A$:

\begin{align} 
K_A &= P_B \diamond b \\
&= b^{-1} \cdot P_B \cdot b \\
&= d \cdot e \cdot b \\
&= h \cdot b \\
K_A &= f\end{align}

For $K_B$: 

\begin{align} 
K_B &= P_A \diamond c \\
&= c^{-1} \cdot P_A \cdot c \\
&= c \cdot f \cdot c \\
&= e \cdot c \\ 
K_B &= f\end{align}

### Since $K_A = f$ and $K_B = f$, then $K_A = K_B$



(*Warning:* Do not confuse $a \cdot b$ and $a \diamond b$!)

## 2-4. EZ CDH [4 pts]

One of the earliest developments in public-key cryptography was the *Diffie–Hellman key exchange protocol*.
In a nutshell, Diffie–Hellman works like this:

1. Alice and Bob agree to work in a cyclic group $G$ and use the generator $g \in G$.
   For our example here, suppose $G = \mathbb{Z}_{23}^*$, the multiplicative group of integers modulo $23$,
   and use $g = 5$ (which is a generator of $\mathbb{Z}_{23}^*$) as the base.

2. Alice chooses a secret integer uniformly at random, say $a = 4$, then sends Bob $A = g^a$ (in $\mathbb{Z}_{p}^*$).
    
   Here, $A = 5^4 = 4$ (in $\mathbb{Z}_{23}^*$).
   
3. Bob chooses a secret integer uniformly at random, say $b = 3$, then sends Alice $B = g^b$ (in $\mathbb{Z}_{p}^*$).
    
   Here, $B = 5^3 = 10$ (in $\mathbb{Z}_{23}^*$).

4. Alice computes $s = B^a$(in $\mathbb{Z}_{p}^*$).

   $s = 10^4 = 18$ (in $\mathbb{Z}_{p}^*$)
   
4. Bob computes $s = A^b$ (in $\mathbb{Z}_{p}^*$).

   $s = 4^3 = 18$(in $\mathbb{Z}_{p}^*$)
   
5. Alice and Bob now share a secret, which is $s = 18$.

It relies on the yet unproven hardness of the computational Diffie–Hellman (CDH) problem:

> Let $G$ be a (multiplicative) cyclic group. Given a generator $g$, and elements $g^a$ and $g^b$, find $g^{ab}$.

Typically you would use $\mathbb{Z}_p^*$ as your cyclic group.
Now let's say Alice and Bob decided to do Diffie–Hellman using an *additive* group $(\mathbb{Z}_p, +)$ instead.
Recall that in an additive group, the associated operation is addition (instead of multiplication), and group exponentiation works like multiplying by a constant.
So $g h$ turns into $g + h$ and $g^x$ turns into $x \cdot g$.

What could go wrong?
It turns out, the CDH problem becomes **trivial** for $(\mathbb{Z}_p, +)$, the set of integers mod $p$ under addition, so if Diffie–Hellman is done using this group, then the secret shared key $s$ can be easily recovered!



Suppose you have intercepted the values of $p$, $g$, $A = a \cdot g$, and $B = b \cdot g$ (remember these are all publicly known, unlike $a$ and $b$), each defined as follows:

In [20]:
A = 20782609745887880863084861164817249644924468778279537828708132307538860361752107659413744391033231295802952551707126922034793630858200019397812144632753634677474169838613791886799852405274342993668918505580248151348428078538557152645014288314096818927299400546703691486452341016537368653345507885348897354018203545337816947203535525392237362546715226227331771463778302927380311598270422291713440018223431224064432559679726313089126054258132462588337710509735129504402910939762843747281910591718386954799004365155651056783262429118602808290058749065586300403862436089590093386848556545179733658861380177829783058929436

In [21]:
p = 26252224380041047747122591194022887149606289495583988602108153223076105239345713403708414193367723342835700767028826621933301528577621675004701754797657444383241868615529900860134429255439131226429056482990442958014081622857451797053181679889832593668643778150520532513821598194302936931987644628206366307016154187413361996418218134120157336904570688208025410591110777302835970098122474666165460550417030400590807644619120346173596400270510603471434129931967590413985206434931078061344456864191168422370416631720528602666860196571809429832072103925681951495398210296474633895268747642683340902741844837855876639885661

In [22]:
B = 15652288550587494861877284061022581048876566461069217909821769631889690057460105326867911142783886861327646427717099133808430792895834938107266178689476692623686149419620572377238190972372792877990229731096982385952118208285712974512553297363466052295585111005436433385888720359053256192304284989071596582442026662910185323452995814587766386534864308324395516118953743865708020444605410498568021948384059886368186871833659783431183873460050303589135767256406110172836808069604572530592421554590407545064725362034890296313995545962574180455654390941548009781096282021384272056883168764941369041848165926711813244361023

In [23]:
g = 10358660391518567616235732436992715220505399810845827441257333108393443171959889069099715138044684141838118325500386741093800010558270378466883199825399330676381466009675928281488401043191332587062637256044696977384105044684466794686621421205589974180482443673243898688602649369028804054011906974785078294135687717039605783356266531235356135655782238895109470371160490304119821561072702573864453584670985173202273982561449531592431520177035322176963872128032236270259466337044311497471533890454091926268065712857277627251241513856846766590882663345959517800443199041098980907470045584969568016546154101486496241468201

Find out what the shared secret $s = (ab)\cdot g$ is, *without* resorting to brute force.

In [24]:
Z = Zmod(p)
a = Z(A/g); a

59834585443026158967062287822398840779230978513988922002496760057094880988024122185249611571240546775565820573681071581512779064448811694109324657428693651014357378448313252346445313848413044341663253373739992764466429599215069066511441997283855433881406697829680701607821177394908844796400974801057278490017383390631767805469906657262672921105629560391174521214752340987799695743867330195144219814280083669967779424361246937345769326838796614390654417837000925253944720538554263645037290955579756953

In [25]:
b = Z(B/g); b

52296988080186291989497679357471386834039958572694307939464075078242760694475758400025855653791169459191371528624723619755808593790202003262979286153406643219840143912265937601337270790535499776707603973911816005158841037890719090705092774439465414152839916478309163071068580277631457358816286358996994942786663416727615533667433068311204953825108228477743856471491844030189963191436075411785672234175409327339550073567600303170586079459429851564270985543655459489165630045005517335816171335848393482

In [26]:
s = Z((a*b) * g); s

2550834175460151232475225102498116410526397584379687765072333637768728614160405520702944601006855407289655105344039920389230603832691118383126762465221838309559950593019786927746313328878458107013341164958661619654546677310796859986215970146650864823464476335027781913693140504074056287333028124212542430118278676044379468927533114473508447864390506461877873302542853292106042492394494438509804236270036121539574303913192107460131761620204701486350256749010840981427244056365987878227684702087114387079030804310548546867375029021886736749142730379864582031554089635947841170488598820652371636579960535209216051580806

In [28]:
#Checking
s_1 = Z(B * a);
s_2 = Z(A * b);
assert(s_1 == s_2), "Secret not Shared"

In [29]:
print(f"The secret is {s}")

The secret is 2550834175460151232475225102498116410526397584379687765072333637768728614160405520702944601006855407289655105344039920389230603832691118383126762465221838309559950593019786927746313328878458107013341164958661619654546677310796859986215970146650864823464476335027781913693140504074056287333028124212542430118278676044379468927533114473508447864390506461877873302542853292106042492394494438509804236270036121539574303913192107460131761620204701486350256749010840981427244056365987878227684702087114387079030804310548546867375029021886736749142730379864582031554089635947841170488598820652371636579960535209216051580806
